In [1]:
print("fix asyncio")

fix asyncio


In [2]:
import asyncio

class AsyncCounter:
    def __init__(self, name, n=3,sleep=2):
        self.n = n
        self.name = name
        self.sleep = sleep

    def __aiter__(self):
        self.i = 0
        return self

    async def __anext__(self):
        if self.i >= self.n:
            raise StopAsyncIteration
        await asyncio.sleep(self.sleep)
        self.i += 1
        return self.i

In [3]:
from datetime import datetime


async def listen_all(counters):
    state = {c.name: None for c in counters}
    queue = asyncio.Queue()
    done={c.name: False for c in counters}
    async def consume(counter: AsyncCounter):
        async for value in counter:
            state[counter.name] = value
            await queue.put((counter.name, value))
        print(f"{counter.name} DONE")
        done[counter.name] = True
        
    # стартуем обработчики
    tasks = [asyncio.create_task(consume(c)) for c in counters]

    # агрегатор
    while True:
        name, value = await queue.get()
        state[name] = value

        print(datetime.now()," Updated state:", state)

        # завершаем, если все потоки закончились
        if all(done.values()):
            print("ALL DONE!")
            break

    # ожидание всех задач
    await asyncio.gather(*tasks)
    return state

In [6]:
async def main():
    counters = [
        AsyncCounter("N9_S1", 9,1),
        AsyncCounter("N4_S2", 4,2.5),
        AsyncCounter("N3_S3", 3,3.3),
    ]
    result = await listen_all(counters)
    print("Final:", result)

In [7]:
await main()

2025-11-22 11:45:45.882548  Updated state: {'N9_S1': 1, 'N4_S2': None, 'N3_S3': None}
2025-11-22 11:45:46.883587  Updated state: {'N9_S1': 2, 'N4_S2': None, 'N3_S3': None}
2025-11-22 11:45:47.382283  Updated state: {'N9_S1': 2, 'N4_S2': 1, 'N3_S3': None}
2025-11-22 11:45:47.885021  Updated state: {'N9_S1': 3, 'N4_S2': 1, 'N3_S3': None}
2025-11-22 11:45:48.182579  Updated state: {'N9_S1': 3, 'N4_S2': 1, 'N3_S3': 1}
2025-11-22 11:45:48.887542  Updated state: {'N9_S1': 4, 'N4_S2': 1, 'N3_S3': 1}
2025-11-22 11:45:49.883616  Updated state: {'N9_S1': 4, 'N4_S2': 2, 'N3_S3': 1}
2025-11-22 11:45:49.887855  Updated state: {'N9_S1': 5, 'N4_S2': 2, 'N3_S3': 1}
2025-11-22 11:45:50.889057  Updated state: {'N9_S1': 6, 'N4_S2': 2, 'N3_S3': 1}
2025-11-22 11:45:51.483872  Updated state: {'N9_S1': 6, 'N4_S2': 2, 'N3_S3': 2}
2025-11-22 11:45:51.890573  Updated state: {'N9_S1': 7, 'N4_S2': 2, 'N3_S3': 2}
2025-11-22 11:45:52.384335  Updated state: {'N9_S1': 7, 'N4_S2': 3, 'N3_S3': 2}
2025-11-22 11:45:52.89